In [4]:
import pandas as pd

def process_postal_data(postal_csv_path: str, target_prefectures: list) -> pd.DataFrame:
    """
    郵便番号データを読み込み、指定された県のみを抽出して
    {都道府県名, 市区町村名, 町域名} の DataFrame を返す
    """
    # エンコーディングは環境やファイルによって異なる可能性あり
    # utf-8 / utf-8-sig / cp932 / shift_jis などを適宜指定
    df = pd.read_csv(postal_csv_path, encoding='utf-8', header=None, dtype=str)
    
    # KEN_ALL.CSV (15列) の想定カラム
    df.columns = [
        'JISX0401_2',             # 0: 全国地方公共団体コード
        'old_postal',             # 1: （旧）郵便番号（5桁）
        'postal',                 # 2: 郵便番号（7桁）
        'pref_kana',             # 3: 都道府県名カナ
        'city_kana',             # 4: 市区町村名カナ
        'area_kana',             # 5: 町域名カナ
        '都道府県名',             # 6: 都道府県名（漢字）
        '市区町村名',             # 7: 市区町村名（漢字）
        '町域名',                 # 8: 町域名（漢字）
        'multi_postal_flag',     # 9: 一町域が二以上の郵便番号で表される場合の表示
        'splitted_kouaza_flag',  # 10: 小字毎に番地が起番されている町域の表示
        'has_chome_flag',        # 11: 丁目を有する町域の場合の表示
        'multi_town_flag',       # 12: 一つの郵便番号で二以上の町域を表す場合
        'update_flag',           # 13: 更新の表示 (0,1,2)
        'update_reason'          # 14: 変更理由 (0～6)
    ]
    
    # 指定県だけ抽出 (漢字の都道府県名列が '都道府県名')
    df_filtered = df[df['都道府県名'].isin(target_prefectures)]
    
    # 重複削除 (市区町村名 + 町域名 が重複している行をまとめる)
    df_filtered = df_filtered[['都道府県名','市区町村名','町域名']].drop_duplicates()
    return df_filtered

# 例: 静岡・山梨・神奈川の町域リストを作る
target_prefs = ['静岡県', '山梨県', '神奈川県']
df_areas = process_postal_data('utf_ken_all.csv', target_prefs)

print(df_areas.head(10))
print(len(df_areas), 'rows')


      都道府県名   市区町村名         町域名
41235  神奈川県  横浜市鶴見区  以下に掲載がない場合
41236  神奈川県  横浜市鶴見区         朝日町
41237  神奈川県  横浜市鶴見区         安善町
41238  神奈川県  横浜市鶴見区        市場上町
41239  神奈川県  横浜市鶴見区        市場下町
41240  神奈川県  横浜市鶴見区       市場東中町
41241  神奈川県  横浜市鶴見区       市場西中町
41242  神奈川県  横浜市鶴見区      市場富士見町
41243  神奈川県  横浜市鶴見区       市場大和町
41244  神奈川県  横浜市鶴見区         潮田町
6199 rows


In [5]:
import requests
import time
import yaml

def geocode_from_gsi(pref: str, city: str, area: str) -> dict:
    """国土地理院APIで住所(都道府県+市区町村+町域)を検索し、座標を返す"""
    address_str = f"{pref}{city}{area}"
    url = f"https://msearch.gsi.go.jp/address-search/AddressSearch?q={address_str}"
    try:
        r = requests.get(url)
        data = r.json()
        if data:
            coords = data[0]['geometry']['coordinates']  # [lon, lat]
            return {'longitude': coords[0], 'latitude': coords[1]}
        else:
            return {'longitude': None, 'latitude': None}
    except:
        return {'longitude': None, 'latitude': None}

def create_geo_cache(df_areas: pd.DataFrame, output_yaml='areas_with_coords.yml'):
    """
    町域の一覧DFから座標を一括取得し、
    {
      "静岡県": {
        "富士宮市": {
          "佐折": {"longitude": 138.xxx, "latitude": 35.xxx},
          ...
        },
        ...
      },
      "山梨県": ...
      "神奈川県": ...
    }
    のような形で yaml に保存する。
    """
    cache_dict = {}
    
    for idx, row in df_areas.iterrows():
        pref = row['都道府県名']
        city = row['市区町村名']
        area = row['町域名']
        
        if pref not in cache_dict:
            cache_dict[pref] = {}
        if city not in cache_dict[pref]:
            cache_dict[pref][city] = {}
        
        coords = geocode_from_gsi(pref, city, area)
        cache_dict[pref][city][area] = coords
        
        # レートリミットを考慮
        time.sleep(1)
    
    with open(output_yaml, 'w', encoding='utf-8') as f:
        yaml.dump(cache_dict, f, allow_unicode=True)
    print(f"座標キャッシュを作成しました: {output_yaml}")

# 実行
create_geo_cache(df_areas, 'areas_with_coords.yml')


座標キャッシュを作成しました: areas_with_coords.yml
